In [40]:
import sys
import os
import numpy as np
from collections import Counter
from collections import defaultdict
import json
from nltk.metrics import distance

In [41]:
def read_source(directory_path):
    source_arr = []
    for root, dirs, files in os.walk(directory_path):
        for f in files:
            with open(directory_path + "/" + f, "r") as source:
                s = source.read()
                source_arr.append((f, f.split('_')[0], s))
    return source_arr

In [42]:
def get_file_extensions():
    path = os.path.expanduser("~/Dropbox/classes/Fall 2017/project/data/{}/source/".format("AAN"))
    arr = read_source(path)
    print Counter([a[0].split('.')[-1] for a in arr])
    
get_file_extensions()

Counter({'py': 10138, 'java': 4974, 'md': 1435, 'cpp': 902, 'c': 626, 'js': 376, 'pl': 73, 'r': 4})


In [43]:
def read_json_file():
    path = os.path.expanduser("~/Dropbox/classes/Fall 2017/project/data/AAN/source_features.json")
    with open(path, "r") as f:
        file_list = json.loads(f.read())
    
    return file_list
    

In [51]:
def search_for_name(keyword, json_dict, type_filter = None):
    matches = []
    for file_name in json_dict.keys():
        for line in json_dict[file_name]:
            right_type = (type_filter is None or line["type"] == type_filter)
            if line["name"] == keyword and right_type:
                matches.append((file_name, line))
    return matches
    
    

In [67]:
def search_for_docstring(keyword, json_dict, type_filter = None):
    matches = []
    for file_name in json_dict.keys():
        for line in json_dict[file_name]:
            right_type = (type_filter is None or line["type"] == type_filter)
            if keyword in line["docstring"] and right_type:
                matches.append((file_name, line))
    return matches

In [45]:
def read_source(file_name):
    try:
        with open(file_name, "r") as f:
            source = f.read()
        return source
    except Exception:
        return ""
    

In [46]:
def print_source(file_name, line_start, line_end):
    source = read_source(file_name)
    return "\n".join(source.split("\n")[line_start - 1 :line_end])
        

In [47]:
def get_indentation(line_string):
    whitespace = [" ", "\t"]
    indentation = ""
    for char in line_string:
        if char in whitespace:
            indentation += char
        else:
            break
    return indentation

In [48]:
def infer_end_line(file_name, line_dict):
    source = read_source(file_name)
    lines = source.split("\n")
    
    target_indentation = get_indentation(lines[line_dict["line"] - 1])
    i = line_dict["line"]
    while i < len(lines):
        line = lines[i]
        if (get_indentation(line) == target_indentation) and len(line) > 0:
            if i == line_dict["line"]:
                return i
            return i - 1
        i += 1
    return i
        
    
    

In [56]:
def find_matches_by_name(json_dict, search_term, type_filter=None):
    matches = search_for_name(search_term, json_dict, type_filter)
    if len(matches):
        end =  infer_end_line(matches[0][0], matches[0][1])
        return print_source(matches[0][0], matches[0][1]["line"], end)
    else:
        return "No Matches Found."

    

In [68]:
def find_matches_by_docstring(json_dict, search_term, type_filter=None):
    matches = search_for_docstring(search_term, json_dict, type_filter)
    if len(matches):
        end =  infer_end_line(matches[0][0], matches[0][1])
        return print_source(matches[0][0], matches[0][1]["line"], end)
    else:
        return "No Matches Found."

In [57]:
json_dict = read_json_file()

In [73]:
# print find_matches_by_name(json_dict, "sofax", "function")
print find_matches_by_docstring(json_dict, "softmax")

  def SoftmaxEval(self, sess, model, num_steps):
    """Evaluate a model in softmax mode.

    Adds char, word recall and sequence error rate events to the sw summary
    writer, and returns them as well
    TODO(rays) Add LogisticEval.
    Args:
      sess:  A tensor flow Session.
      model: The model to run in the session. Requires a VGSLImageModel or any
        other class that has a using_ctc attribute and a RunAStep(sess) method
        that reurns a softmax result with corresponding labels.
      num_steps: Number of steps to evaluate for.
    Returns:
      ErrorRates named tuple.
    Raises:
      ValueError: If an unsupported number of dimensions is used.
    """
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    # Run the requested number of evaluation steps, gathering the outputs of the
    # softmax and the true labels of the evaluation examples.
    total_label_counts = ec.ErrorCounts(0, 0, 0, 0)
    total_word_coun